In [ ]:
pip install transformers

In [ ]:
import os
import re
import string
import json
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
data = pd.read_excel('L5.xlsx')

In [ ]:
data.head()

,Text_files,Activating invasion and metastasis--Invasion,Inducing angiogenesis--By deregulating angiogenesis--Angiogenic factors,Evading growth suppressors--By deregulating cell cycle checkpoints--,Evading growth suppressors--By deregulating cell cycle checkpoints--Cell cycle,Evading growth suppressors--By evading contact inhibition,Avoiding immune destruction--Immune response,Genomic instability and mutation--Mutation,Tumor promoting inflammation--Inflammation--,Resisting cell death--Apoptosis,Enabling replicative immortality--Immortalization,Avoiding immune destruction--Immunosuppression,Inducing angiogenesis--,Genomic instability and mutation--,Activating invasion and metastasis--,Genomic instability and mutation--DNA damage--Adducts,Genomic instability and mutation--DNA damage--,Sustaining proliferative signaling--Cell cycle,Cellular energetics--Glycolysis/Warburg effect,Sustaining proliferative signaling--,Genomic instability and mutation--DNA repair mechanisms,Cellular energetics--,Sustaining proliferative signaling--Growth factors growth promoting signals--,Tumor promoting inflammation--Immune response,Resisting cell death--Necrosis,Genomic instability and mutation--DNA damage--Strand breaks,Sustaining proliferative signaling--Receptors,Tumor promoting inflammation--,Sustaining proliferative signaling--Growth factors growth promoting signals--Downstream signaling,Inducing angiogenesis--By deregulating angiogenesis,NULL,Activating invasion and metastasis--Metastasis,Resisting cell death--,Enabling replicative immortality--,Avoiding immune destruction--,Tumor promoting inflammation--Inflammation--Oxidative stress,Enabling replicative immortality--Senescence,Resisting cell death--Autophagy
0,Background .\nOver the past ten years oncologi...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Treatment for glioblastoma multiforme includes...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Prostate cancer ( PCA ) is the most common inv...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Splenic haemangiosarcomas ( HSAs ) from 122 do...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Application of adenovirus vectors ( Adv ) in m...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
data.shape

(1851, 38)

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
train.head()

,Text_files,Activating invasion and metastasis--Invasion,Inducing angiogenesis--By deregulating angiogenesis--Angiogenic factors,Evading growth suppressors--By deregulating cell cycle checkpoints--,Evading growth suppressors--By deregulating cell cycle checkpoints--Cell cycle,Evading growth suppressors--By evading contact inhibition,Avoiding immune destruction--Immune response,Genomic instability and mutation--Mutation,Tumor promoting inflammation--Inflammation--,Resisting cell death--Apoptosis,Enabling replicative immortality--Immortalization,Avoiding immune destruction--Immunosuppression,Inducing angiogenesis--,Genomic instability and mutation--,Activating invasion and metastasis--,Genomic instability and mutation--DNA damage--Adducts,Genomic instability and mutation--DNA damage--,Sustaining proliferative signaling--Cell cycle,Cellular energetics--Glycolysis/Warburg effect,Sustaining proliferative signaling--,Genomic instability and mutation--DNA repair mechanisms,Cellular energetics--,Sustaining proliferative signaling--Growth factors growth promoting signals--,Tumor promoting inflammation--Immune response,Resisting cell death--Necrosis,Genomic instability and mutation--DNA damage--Strand breaks,Sustaining proliferative signaling--Receptors,Tumor promoting inflammation--,Sustaining proliferative signaling--Growth factors growth promoting signals--Downstream signaling,Inducing angiogenesis--By deregulating angiogenesis,NULL,Activating invasion and metastasis--Metastasis,Resisting cell death--,Enabling replicative immortality--,Avoiding immune destruction--,Tumor promoting inflammation--Inflammation--Oxidative stress,Enabling replicative immortality--Senescence,Resisting cell death--Autophagy
344,Several basic leucine zipper ( B-ZIP ) transcr...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1444,It is well established that blocking the inter...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
408,Genipin is a metabolite of geniposide isolated...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
49,BACKGROUND Modulation of the expression of ret...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
342,Metastatic breast tumors undergo epithelial-to...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test.head()

,Text_files,Activating invasion and metastasis--Invasion,Inducing angiogenesis--By deregulating angiogenesis--Angiogenic factors,Evading growth suppressors--By deregulating cell cycle checkpoints--,Evading growth suppressors--By deregulating cell cycle checkpoints--Cell cycle,Evading growth suppressors--By evading contact inhibition,Avoiding immune destruction--Immune response,Genomic instability and mutation--Mutation,Tumor promoting inflammation--Inflammation--,Resisting cell death--Apoptosis,Enabling replicative immortality--Immortalization,Avoiding immune destruction--Immunosuppression,Inducing angiogenesis--,Genomic instability and mutation--,Activating invasion and metastasis--,Genomic instability and mutation--DNA damage--Adducts,Genomic instability and mutation--DNA damage--,Sustaining proliferative signaling--Cell cycle,Cellular energetics--Glycolysis/Warburg effect,Sustaining proliferative signaling--,Genomic instability and mutation--DNA repair mechanisms,Cellular energetics--,Sustaining proliferative signaling--Growth factors growth promoting signals--,Tumor promoting inflammation--Immune response,Resisting cell death--Necrosis,Genomic instability and mutation--DNA damage--Strand breaks,Sustaining proliferative signaling--Receptors,Tumor promoting inflammation--,Sustaining proliferative signaling--Growth factors growth promoting signals--Downstream signaling,Inducing angiogenesis--By deregulating angiogenesis,NULL,Activating invasion and metastasis--Metastasis,Resisting cell death--,Enabling replicative immortality--,Avoiding immune destruction--,Tumor promoting inflammation--Inflammation--Oxidative stress,Enabling replicative immortality--Senescence,Resisting cell death--Autophagy
233,A growing number of studies have demonstrated ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1567,BACKGROUND/AIMS Adenocarcinoma of the gallblad...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
486,This article describes evaluation of plasma me...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
799,"Each year , more than 700,000 people undergo c...",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1233,As part of our program to develop breast cance...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
print(train.shape)
print(test.shape)

(1665, 38)
(186, 38)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
length = 0
for row in data['Text_files']:
  if length < len(row):
    length = len(row)
print(length)

4162


Cleaning the texts

In [ ]:
def clean_text(text):
    ''' Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

In [ ]:
def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shalsham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.tokenize import TreebankWordTokenizer

In [ ]:
stop_words = list(stopwords.words('english'))

In [ ]:
def remove_stop_words(text):
    text_tokenized = text.split()
    filtered_text = [w for w in text_tokenized if not w in stop_words]
    filtered_text = " ".join(filtered_text)
    
    return filtered_text

In [ ]:
def stemming_text(text):
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    porter = nltk.PorterStemmer()
    stemmed_text = [porter.stem(t) for t in tokens]
    final_text = " ".join(stemmed_text)
    return final_text


def lemmatizing_text(text):
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    wnl = nltk.WordNetLemmatizer()
    
    lemmatized_text = [wnl.lemmatize(t) for t in tokens]
    final_text = " ".join(lemmatized_text)
    return final_text

In [ ]:
def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = remove_stop_words(text)
    text = remove_space(text)
    # text = stemming_text(text)
    # text = lemmatizing_text(text)
    return text

In [ ]:
def clean(df, text_field):
    df['cleaned_text'] = df[text_field].apply(lambda x: text_preprocessing_pipeline(str(x)))
    return df

In [ ]:
clean(train, 'Text_files')

,Text_files,Activating invasion and metastasis--Invasion,Inducing angiogenesis--By deregulating angiogenesis--Angiogenic factors,Evading growth suppressors--By deregulating cell cycle checkpoints--,Evading growth suppressors--By deregulating cell cycle checkpoints--Cell cycle,Evading growth suppressors--By evading contact inhibition,Avoiding immune destruction--Immune response,Genomic instability and mutation--Mutation,Tumor promoting inflammation--Inflammation--,Resisting cell death--Apoptosis,Enabling replicative immortality--Immortalization,Avoiding immune destruction--Immunosuppression,Inducing angiogenesis--,Genomic instability and mutation--,Activating invasion and metastasis--,Genomic instability and mutation--DNA damage--Adducts,Genomic instability and mutation--DNA damage--,Sustaining proliferative signaling--Cell cycle,Cellular energetics--Glycolysis/Warburg effect,Sustaining proliferative signaling--,Genomic instability and mutation--DNA repair mechanisms,Cellular energetics--,Sustaining proliferative signaling--Growth factors growth promoting signals--,Tumor promoting inflammation--Immune response,Resisting cell death--Necrosis,Genomic instability and mutation--DNA damage--Strand breaks,Sustaining proliferative signaling--Receptors,Tumor promoting inflammation--,Sustaining proliferative signaling--Growth factors growth promoting signals--Downstream signaling,Inducing angiogenesis--By deregulating angiogenesis,NULL,Activating invasion and metastasis--Metastasis,Resisting cell death--,Enabling replicative immortality--,Avoiding immune destruction--,Tumor promoting inflammation--Inflammation--Oxidative stress,Enabling replicative immortality--Senescence,Resisting cell death--Autophagy,cleaned_text
344,Several basic leucine zipper ( B-ZIP ) transcr...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,several basic leucine zipper b zip transcripti...
1444,It is well established that blocking the inter...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,well established blocking interaction egfr gro...
408,Genipin is a metabolite of geniposide isolated...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,genipin metabolite geniposide isolated extract...
49,BACKGROUND Modulation of the expression of ret...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,background modulation expression retinoic acid...
342,Metastatic breast tumors undergo epithelial-to...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,metastatic breast tumors undergo epithelial me...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,OBJECTIVE Hepatocellular carcinoma ( HCC ) is ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,objective hepatocellular carcinoma hcc highly ...
1294,Cav-1 ( -/- ) deficient stromal cells are a ne...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,cav deficient stromal cells new genetic model ...
860,The NOTCH signaling pathway plays important ro...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,notch signaling pathway plays important role d...
1459,Imiquimod is a synthetic compound with antitum...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,imiquimod synthetic compound antitumor propert...


In [ ]:
clean(test, 'Text_files')

,Text_files,Activating invasion and metastasis--Invasion,Inducing angiogenesis--By deregulating angiogenesis--Angiogenic factors,Evading growth suppressors--By deregulating cell cycle checkpoints--,Evading growth suppressors--By deregulating cell cycle checkpoints--Cell cycle,Evading growth suppressors--By evading contact inhibition,Avoiding immune destruction--Immune response,Genomic instability and mutation--Mutation,Tumor promoting inflammation--Inflammation--,Resisting cell death--Apoptosis,Enabling replicative immortality--Immortalization,Avoiding immune destruction--Immunosuppression,Inducing angiogenesis--,Genomic instability and mutation--,Activating invasion and metastasis--,Genomic instability and mutation--DNA damage--Adducts,Genomic instability and mutation--DNA damage--,Sustaining proliferative signaling--Cell cycle,Cellular energetics--Glycolysis/Warburg effect,Sustaining proliferative signaling--,Genomic instability and mutation--DNA repair mechanisms,Cellular energetics--,Sustaining proliferative signaling--Growth factors growth promoting signals--,Tumor promoting inflammation--Immune response,Resisting cell death--Necrosis,Genomic instability and mutation--DNA damage--Strand breaks,Sustaining proliferative signaling--Receptors,Tumor promoting inflammation--,Sustaining proliferative signaling--Growth factors growth promoting signals--Downstream signaling,Inducing angiogenesis--By deregulating angiogenesis,NULL,Activating invasion and metastasis--Metastasis,Resisting cell death--,Enabling replicative immortality--,Avoiding immune destruction--,Tumor promoting inflammation--Inflammation--Oxidative stress,Enabling replicative immortality--Senescence,Resisting cell death--Autophagy,cleaned_text
233,A growing number of studies have demonstrated ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,growing number studies demonstrated associatio...
1567,BACKGROUND/AIMS Adenocarcinoma of the gallblad...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,background aims adenocarcinoma gallbladder hig...
486,This article describes evaluation of plasma me...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,article describes evaluation plasma membrane f...
799,"Each year , more than 700,000 people undergo c...",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"year , , people undergo cancer surgery united ..."
1233,As part of our program to develop breast cance...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,part program develop breast cancer specific th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,Many tumors contain heterogeneous populations ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,many tumors contain heterogeneous populations ...
1078,Colon cancer is a leading cause of morbidity a...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,colon cancer leading cause morbidity mortality...
124,IL-16 is a ligand and chemotactic factor for C...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,il ligand chemotactic factor cells .il inhibit...
733,BACKGROUND The uneven distribution of recombin...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,background uneven distribution recombination a...


In [ ]:
def change_dtype(value):
    try:
        return int(value)
    except ValueError:
        try:
            return float(value)
        except ValueError:
            return value

In [ ]:
target_cols = [col for col in train.columns if col not in ['Text_files', 'cleaned_text']]
target_cols

['Activating invasion and metastasis--Invasion',
 'Inducing angiogenesis--By deregulating angiogenesis--Angiogenic factors',
 'Evading growth suppressors--By deregulating cell cycle checkpoints--',
 'Evading growth suppressors--By deregulating cell cycle checkpoints--Cell cycle',
 'Evading growth suppressors--By evading contact inhibition',
 'Avoiding immune destruction--Immune response',
 'Genomic instability and mutation--Mutation',
 'Tumor promoting inflammation--Inflammation--',
 'Resisting cell death--Apoptosis',
 'Enabling replicative immortality--Immortalization',
 'Avoiding immune destruction--Immunosuppression',
 'Inducing angiogenesis--',
 'Genomic instability and mutation--',
 'Activating invasion and metastasis--',
 'Genomic instability and mutation--DNA damage--Adducts',
 'Genomic instability and mutation--DNA damage--',
 'Sustaining proliferative signaling--Cell cycle',
 'Cellular energetics--Glycolysis/Warburg effect',
 'Sustaining proliferative signaling--',
 'Genomic i

In [ ]:
for column in target_cols:
    train.loc[:, column] = train[column].apply(change_dtype)

In [ ]:
for column in target_cols:
    test.loc[:, column] = test[column].apply(change_dtype)

In [ ]:
MAX_LEN = 0
for row in train['cleaned_text']:
  if MAX_LEN < len(row):
    MAX_LEN = len(row)

print(MAX_LEN)

2963


In [ ]:
MAX_LEN1 = 0
for row in test['cleaned_text']:
  if MAX_LEN1 < len(row):
    MAX_LEN1 = len(row)

print(MAX_LEN1)

2434


In [ ]:
MIN_LEN = 10000
for row in train['cleaned_text']:
  if MIN_LEN > len(row):
    MIN_LEN = len(row)

print(MIN_LEN)

253


In [ ]:
avg_len = 0
for row in train['cleaned_text']:
  avg_len += len(row)
print(avg_len  / len(train['cleaned_text']))

1185.4936936936938


In [ ]:
larger_than_512 = 0
for row in train['cleaned_text']:
  if len(row) > 1500:
    larger_than_512 += 1
print(larger_than_512)
print(len(train['cleaned_text']) - larger_than_512)

236
1429


In [ ]:
train.reset_index(inplace=True)
train.head()

,index,Text_files,Activating invasion and metastasis--Invasion,Inducing angiogenesis--By deregulating angiogenesis--Angiogenic factors,Evading growth suppressors--By deregulating cell cycle checkpoints--,Evading growth suppressors--By deregulating cell cycle checkpoints--Cell cycle,Evading growth suppressors--By evading contact inhibition,Avoiding immune destruction--Immune response,Genomic instability and mutation--Mutation,Tumor promoting inflammation--Inflammation--,Resisting cell death--Apoptosis,Enabling replicative immortality--Immortalization,Avoiding immune destruction--Immunosuppression,Inducing angiogenesis--,Genomic instability and mutation--,Activating invasion and metastasis--,Genomic instability and mutation--DNA damage--Adducts,Genomic instability and mutation--DNA damage--,Sustaining proliferative signaling--Cell cycle,Cellular energetics--Glycolysis/Warburg effect,Sustaining proliferative signaling--,Genomic instability and mutation--DNA repair mechanisms,Cellular energetics--,Sustaining proliferative signaling--Growth factors growth promoting signals--,Tumor promoting inflammation--Immune response,Resisting cell death--Necrosis,Genomic instability and mutation--DNA damage--Strand breaks,Sustaining proliferative signaling--Receptors,Tumor promoting inflammation--,Sustaining proliferative signaling--Growth factors growth promoting signals--Downstream signaling,Inducing angiogenesis--By deregulating angiogenesis,NULL,Activating invasion and metastasis--Metastasis,Resisting cell death--,Enabling replicative immortality--,Avoiding immune destruction--,Tumor promoting inflammation--Inflammation--Oxidative stress,Enabling replicative immortality--Senescence,Resisting cell death--Autophagy,cleaned_text
0,344,Several basic leucine zipper ( B-ZIP ) transcr...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,several basic leucine zipper b zip transcripti...
1,1444,It is well established that blocking the inter...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,well established blocking interaction egfr gro...
2,408,Genipin is a metabolite of geniposide isolated...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,genipin metabolite geniposide isolated extract...
3,49,BACKGROUND Modulation of the expression of ret...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,background modulation expression retinoic acid...
4,342,Metastatic breast tumors undergo epithelial-to...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,metastatic breast tumors undergo epithelial me...


In [ ]:
test.reset_index(inplace=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification

In [ ]:
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 1e-5
#tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-large-cased-v1.1-mnli')
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

In [ ]:
print(len(target_cols))

37


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.cleaned_text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        self.chunksize = 512
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        # inputs = self.tokenizer.encode_plus(
        #     text,
        #     truncation=True,
        #     add_special_tokens=True,
        #     max_length=self.max_len,
        #     padding='max_length',
        #     return_token_type_ids=True
        # )
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens = False,
            truncation=False,
            return_token_type_ids = True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors = 'pt'
        )
        # ids = inputs['input_ids']
        # mask = inputs['attention_mask']
        input_id_chunks = list(inputs['input_ids'][0].split(510))
        mask_chunks = list(inputs['attention_mask'][0].split(510))
        token_type_ids_chunks = list(inputs['token_type_ids'][0].split(510))

        # get required padding length
        for i in range(len(input_id_chunks)):
          input_id_chunks[i] = torch.cat(
              [torch.Tensor([101]), input_id_chunks[i], torch.Tensor([102])]
          )
          mask_chunks[i] = torch.cat(
              [torch.Tensor([1]), mask_chunks[i], torch.Tensor([1])]
          )
          token_type_ids_chunks[i] = torch.cat(
              [torch.Tensor([0]), token_type_ids_chunks[i], torch.Tensor([0])]
          )
          
          pad_len = self.chunksize - input_id_chunks[i].shape[0]
          if pad_len > 0:
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_len)
            ])

            mask_chunks[i] = torch.cat([
                mask_chunks[i], torch.Tensor([0] * pad_len)
            ])

            token_type_ids_chunks[i] = torch.cat([
                token_type_ids_chunks[i], torch.Tensor([0] * pad_len)
            ])

        input_ids = torch.stack(input_id_chunks)
        attention_mask = torch.stack(mask_chunks)
        token_type_ids = torch.stack(token_type_ids_chunks)
        
        # return {
        #     'ids': torch.tensor(ids, dtype=torch.long),
        #     'mask': torch.tensor(mask, dtype=torch.long),
        #     'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        #     'targets': torch.tensor(self.targets[index], dtype=torch.float)
        # }

        # input_dict = {
        #     'input_ids': input_ids.long(),
        #     'attention_mask': attention_mask.long(),
        #     'token_type_ids': token_type_ids.long()
        # }

        return {
            
            # 'input_dict': input_dict,
            'ids': input_ids.long(),
            'mask': attention_mask.long(),
            'token_type_ids': token_type_ids.long(),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_dataset = BERTDataset(train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(test, tokenizer, MAX_LEN)

In [ ]:
train_dataset.text

0       several basic leucine zipper b zip transcripti...
1       well established blocking interaction egfr gro...
2       genipin metabolite geniposide isolated extract...
3       background modulation expression retinoic acid...
4       metastatic breast tumors undergo epithelial me...
                              ...                        
1660    objective hepatocellular carcinoma hcc highly ...
1661    cav deficient stromal cells new genetic model ...
1662    notch signaling pathway plays important role d...
1663    imiquimod synthetic compound antitumor propert...
1664    mesotheliomas malignant tumors pleural periton...
Name: cleaned_text, Length: 1665, dtype: object

In [ ]:
train_dataset.text

0       several basic leucine zipper b zip transcripti...
1       well established blocking interaction egfr gro...
2       genipin metabolite geniposide isolated extract...
3       background modulation expression retinoic acid...
4       metastatic breast tumors undergo epithelial me...
                              ...                        
1660    objective hepatocellular carcinoma hcc highly ...
1661    cav deficient stromal cells new genetic model ...
1662    notch signaling pathway plays important role d...
1663    imiquimod synthetic compound antitumor propert...
1664    mesotheliomas malignant tumors pleural periton...
Name: cleaned_text, Length: 1665, dtype: object

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False)

In [ ]:
for data in train_loader:
  print(data['ids'].size())

torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])
torch.Size([32, 6, 512])


In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        #self.biobert = AutoModel.from_pretrained('dmis-lab/biobert-large-cased-v1.1')
        #self.biobert = AutoModelForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
        #self.biobert = AutoModel.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
        # self.biobert = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-large-cased-v1.1-mnli")
        self.biobert = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,37)
    
#     def forward(self, ids, mask, token_type_ids):
#         _, features = self.biobert(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
# #         output_2 = self.l2(output_1)

#         output = self.fc(**features)
#         return output


    def forward(self, ids, mask, token_type_ids):
        _, features = self.biobert(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)

        output = self.fc(features)
        return output

    # def forward(self, ids, mask, token_type_ids):
    #   final_output = torch.zeros(37)
      
    #   for i in range(32):
    #     for j in range(6):
    #       _,current_output = self.biobert(ids[i][j], attention_mask = mask[i][j], token_type_ids = token_type_ids[i][j], return_dict=False)

    #     # _, features = self.biobert(ids[i], attention_mask = mask[i], token_type_ids = token_type_ids[i], return_dict=False)

    #     # current_output = self.fc(features)
    #     # final_output = torch.tensor([1. if final_output[i] or current_output[i] == 1. else 0. for i in range(37)]) 
    #   return final_output

    # def forward(self, input_dict):
    #   output = self.fc(**input_dict)
    #   return output

model = BERTClass()
model.to(device);

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

In [ ]:
for _,data in enumerate(train_loader, 0):
  print(data['ids'])
  print(data['targets'][0])
  break

tensor([[[  101,   176,  1193,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,     0]],

        [[  101,  6457,  3689,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,     0]],

        [[  101,  3055,  2103,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,   102],
         [  101,     0,     0,  ...,     0,     0,  

In [ ]:
list1 = [1,0,1]
list2 = [0,0,0]

list3 = [1 if list1[i] or list2[i] == 1 else 0 for i in range(3)]

print(list3)

[1, 0, 1]


In [ ]:
tesor1 = torch.arange(3)
for i in range(3):
  print(i)
tesor2 = torch.tensor([1 for x in tesor1])
tesor3 = torch.zeros(3)
tesor4 = torch.tensor([1. for x in tesor3])
if tesor4[0] == 1.:
  print("YES")

tesor4.shape

0
1
2
YES


torch.Size([3])

In [ ]:
for _,data in enumerate(test_loader, 0):
  print(data['ids'].size(1))
  print(data['ids'].size())
  break
  

# packed_ids[0] = torch.ones(512)
# print(packed_ids)
for _,data in enumerate(train_loader, 0):
  for i in range(data['ids'].size(1)):
    packed_ids = torch.zeros(32, 512)
    for j in range(data['ids'].size(0)):
      packed_ids[j] = data['ids'][j][i]
    print(packed_ids.size())

In [ ]:
def train_function(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        final_outputs = torch.zeros(32, 37)
        for i in range(data['ids'].size(1)):
          packed_ids = torch.zeros(data['ids'].size(0), 512)
          packed_masks = torch.zeros(data['ids'].size(0), 512)
          packed_token_type_ids = torch.zeros(data['ids'].size(0), 512)
          for j in range(data['ids'].size(0)):
            packed_ids[j] = data['ids'][j][i]
            packed_masks[j] = data['mask'][j][i]
            packed_token_type_ids[j] = data['token_type_ids'][j][i]

          packed_ids = packed_ids.to(device, dtype=torch.long)
          packed_masks = packed_masks.to(device, dtype=torch.long)
          packed_token_type_ids = packed_token_type_ids.to(device, dtype=torch.long)
          outputs = model(packed_ids, packed_masks, packed_token_type_ids)
          # final_output = torch.tensor([1. if final_output[i] or current_output[i] == 1. else 0. for i in range(37)]) 
          final_outputs = torch.tensor([[1. if final_outputs[i][j] or outputs[i][j] == 1. else 0. for j in range(37)] for i in range(32)])
        targets = data['targets']
        loss = loss_fn(final_outputs, targets)
        loss.requires_grad=True

        if _%500 == 0:
          print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # ids = data['ids'].to(device, dtype = torch.long)
        # mask = data['mask'].to(device, dtype = torch.long)
        # token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        # input_dict = data['input_dict']
        # targets = data['targets']
        # outputs = model(ids, mask, token_type_ids)

        # loss = loss_fn(outputs, targets)
        # if _%500 == 0:
        #     print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()

In [ ]:
def train_function(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        final_outputs = torch.zeros(32, 37)
        for i in range(data['ids'].size(1)):
          packed_ids = torch.zeros(data['ids'].size(0), 512)
          packed_masks = torch.zeros(data['ids'].size(0), 512)
          packed_token_type_ids = torch.zeros(data['ids'].size(0), 512)
          for j in range(data['ids'].size(0)):
            packed_ids[j] = data['ids'][j][i]
            packed_masks[j] = data['mask'][j][i]
            packed_token_type_ids[j] = data['token_type_ids'][j][i]

          packed_ids = packed_ids.to(device, dtype=torch.long)
          packed_masks = packed_masks.to(device, dtype=torch.long)
          packed_token_type_ids = packed_token_type_ids.to(device, dtype=torch.long)
          outputs = model(packed_ids, packed_masks, packed_token_type_ids)
          # final_output = torch.tensor([1. if final_output[i] or current_output[i] == 1. else 0. for i in range(37)]) 
          final_outputs = torch.tensor([[1. if final_outputs[i][j] or outputs[i][j] == 1. else 0. for j in range(37)] for i in range(data['ids'].size(0))])
        targets = data['targets']
        loss = loss_fn(final_outputs, targets)
        loss.requires_grad=True

        if _%500 == 0:
          print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # ids = data['ids'].to(device, dtype = torch.long)
        # mask = data['mask'].to(device, dtype = torch.long)
        # token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        # input_dict = data['input_dict']
        # targets = data['targets']
        # outputs = model(ids, mask, token_type_ids)

        # loss = loss_fn(outputs, targets)
        # if _%500 == 0:
        #     print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()

In [ ]:
def train_function(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        fin_outputs=[]

        final_outputs = torch.zeros(data['ids'].size(0), 37)
        for i in range(data['ids'].size(1)):
          packed_ids = torch.zeros(data['ids'].size(0), 512)
          packed_masks = torch.zeros(data['ids'].size(0), 512)
          packed_token_type_ids = torch.zeros(data['ids'].size(0), 512)
          for j in range(data['ids'].size(0)):
            packed_ids[j] = data['ids'][j][i]
            packed_masks[j] = data['mask'][j][i]
            packed_token_type_ids[j] = data['token_type_ids'][j][i]

          packed_ids = packed_ids.to(device, dtype=torch.long)
          packed_masks = packed_masks.to(device, dtype=torch.long)
          packed_token_type_ids = packed_token_type_ids.to(device, dtype=torch.long)
          outputs = model(packed_ids, packed_masks, packed_token_type_ids)
#           print(outputs)
          # final_output = torch.tensor([1. if final_output[i] or current_output[i] == 1. else 0. for i in range(37)]) 
          final_outputs = torch.tensor([[1. if final_outputs[i][j] or outputs[i][j] == 1. else 0. for j in range(37)] for i in range(data['ids'].size(0))])
          fin_outputs.extend(torch.sigmoid(final_outputs).cpu().detach().numpy().tolist())
          print(fin_outputs)
        targets = data['targets']
        loss = loss_fn(final_outputs, targets)
        loss.requires_grad=True

        if _%500 == 0:
          print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # ids = data['ids'].to(device, dtype = torch.long)
        # mask = data['mask'].to(device, dtype = torch.long)
        # token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        # input_dict = data['input_dict']
        # targets = data['targets']
        # outputs = model(ids, mask, token_type_ids)

        # loss = loss_fn(outputs, targets)
        # if _%500 == 0:
        #     print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()

In [ ]:
for epoch in range(2):
    train_function(epoch)

In [ ]:
def test_function():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
            final_outputs = torch.zeros(data['ids'].size(0), 37)
            for i in range(data['ids'].size(1)):
              packed_ids = torch.zeros(data['ids'].size(0), 512)
              packed_masks = torch.zeros(data['ids'].size(0), 512)
              packed_token_type_ids = torch.zeros(data['ids'].size(0), 512)
              for j in range(data['ids'].size(0)):
                packed_ids[j] = data['ids'][j][i]
                packed_masks[j] = data['mask'][j][i]
                packed_token_type_ids[j] = data['token_type_ids'][j][i]

              packed_ids = packed_ids.to(device, dtype=torch.long)
              packed_masks = packed_masks.to(device, dtype=torch.long)
              packed_token_type_ids = packed_token_type_ids.to(device, dtype=torch.long)

              outputs = model(packed_ids, packed_masks, packed_token_type_ids)
              final_outputs = torch.tensor([[1. if final_outputs[i][j] or outputs[i][j] == 1. else 0. for j in range(37)] for i in range(data['ids'].size(0))])

            targets = data['targets'].to(device, dtype = torch.float)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(final_outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
test_function()

In [ ]:
outputs, targets = test_function()
print(outputs[3:10])
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

[[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5

In [ ]:
outputs[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])